# Implementación del Cifrado Afín

El **cifrado afín** es el primero explicado a los neófitos en la Criptografía; ello esencialmente por tres motivos:
* es **conceptualmente "sencillo"**.
* está **basado** en la **aritmética modular** y en el del **[Algoritmo Extendido de Euclides](https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm)**.
* es **extraordinariamente débil** y su ataque mediante el **test Chi-cuadrado** ([Chi-squared test](https://en.wikipedia.org/wiki/Chi-squared_test)) es muy **sencillo** y **eficiente**.

Es por tanto un criptosistema ideal para mostrar al completo el trabajo del criptoanalista.

En nuestro caso podemos añadir un interés más, a saber, que es **fácil de implementar** y **muestra** muy a las claras la conveniencia ---y cómo--- usar eficientemente la **programación dirigida a objetos** (OOP por sus siglas en inglés).  
    

## Módulos

In [ ]:
from math import log, gcd
from itertools import chain, groupby

## Alfabeto y Parámetros del Alfabeto

In [ ]:
alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789;:,.-'
n = len(alphabet)
m = int(log(len(alphabet),10))+1
f = '0{0}d'.format(str(m))
chNum = {v:format(i,f) for i, v in enumerate(alphabet)}
numCh = {v:i for i, v in chNum.items()}

In [ ]:
alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
n = len(alphabet)
m = int(log(len(alphabet),10))+1
f = '0{0}d'.format(str(m))
chNum = {v:format(i,f) for i, v in enumerate(alphabet)}
numCh = {v:i for i, v in chNum.items()}

In [ ]:
alphSpecials = {
            'Á' : 'A',
            'É' : 'E',
            'Í' : 'I',
            'Ó' : 'O',
            'Ú' : 'U',
            'Ä' : 'A',
            'Ë' : 'E',
            'Ï' : 'I',
            'Ö' : 'O',
            'Ü' : 'U',
            'Ñ' : 'GN'
        }

## Normalización

In [ ]:
class Message:
    
    def _flatten(self,listOfLists:list) -> list:
        """Flatten one level of nesting"""
        return chain.from_iterable(listOfLists)
    
    def _rBlanks(self,strng:str) -> str:
        """Removes blanks of a string strng and converts to uppercase"""
        return ''.join(strng.split()).upper()
    
    def _normalize(self,strng:str) -> str:
        """
        Removes blanks spaces of the string 'strng'; then removes accents
        according to 'alphSpecials'. If character 'ñ' occurs in 'strng' then
        'GN' appears in 'accum' as an entry, therefore '_flatten' is needed.
        """
        s = self._rBlanks(strng)
        accum = []
        for ch in s:
            if ch in alphSpecials:
                accum.append(alphSpecials[ch])
            else:
                accum.append(ch)
        return filter(lambda x: x in alphabet,self._flatten(accum))
        # return [c for c in self._flatten(accum) if c in alphabet]

    def __init__(self,strng):
        x = self._normalize(strng)
        self.content = ''.join(x)
        self.length = len(self.content)
        
    def __str__(self):
        return self.content

In [ ]:
A = Message('the secret protects itself.')

In [ ]:
A.content

In [ ]:
print(A)

In [ ]:
A = Message('Niño: seguí al capitán Nemo a lo largo de los corredores y llega-mos al centro del navío.')

In [ ]:
A.content

## Cifrado

Encipher ha sido "mejorada" prescindiendo de la aplicación del algoritmo extendido de Euclides e incluyendo memoización.

In [ ]:
class Encipher(Message):
    
    def _invMod(self,a:int,n:int) -> int:
        """Return multiplicative inverse of a modulo n.
           If the integers a and n are not coprime, then return 0."""
        try:
            x, g = pow(a,-1,n), 1
        except ValueError:
            g = 0
        return int(g==1 and x)

    def _translation(self,c:str,a:int,b:int) -> str:
        return numCh[format((a*int(chNum[c])+b)%len(alphabet),f)]
    
    def __init__(self,strng,a=1,b=0):
        Message.__init__(self,strng)
        self.decimation = a
        self.displacement = b
    
    def affine(self,mode=True):
        """Memoization based code mode: True descifrar"""       
        mem, accum = {}, []
        if mode:
            a = self._invMod(self.decimation,len(alphabet))
            b = -a*self.displacement
        else:
            a, b = self.decimation, self.displacement
        for ch in self.content:
            if ch in mem:
                accum.append(mem[ch])
            else:
                mem[ch]=self._translation(ch,a,b)
                accum.append(mem[ch])
        return ''.join(accum)

In [ ]:
P = Encipher('el secreto se protege a sí mismo',15,24)

In [ ]:
P.content

In [ ]:
P.affine(False)

In [ ]:
E = Encipher('GHIGCTGXAIGPTAXGKGYIOWOIWA',15,24)

In [ ]:
E.affine()

## Ataque

In [ ]:
alphFreq = {
    'A' : 12.53,
    'B' : 1.42,
    'C' : 4.68,
    'D' : 5.86,
    'E' : 13.68,
    'F' : 0.69,
    'G' : 1.01,
    'H' : 0.70,
    'I' : 6.25,
    'J' : 0.44,
    'K' : 0.02,
    'L' : 4.97,
    'M' : 3.15,
    'N' : 6.71,
    'O' : 8.68,
    'P' : 2.51,
    'Q' : 0.88,
    'R' : 6.87,
    'S' : 7.98,
    'T' : 4.63,
    'U' : 3.93,
    'V' : 0.90,
    'W' : 0.01,
    'X' : 0.22,
    'Y' : 0.90,
    'Z' : 0.52
    }

In [ ]:
class ChiSquareAttack(Encipher):
    
    def __init__(self,strng):
        Encipher.__init__(self,strng)
    
    def rfrec(self, strng:str) -> dict:
        return {k:len(list(g))/len(strng) for k, g in groupby(''.join(sorted(strng)))}
    
    def chiSquared(self, strng:str) -> float:
        inventory = dict.fromkeys(alphabet,0)
        inventory.update(self.rfrec(strng))
        chDegree =[(len(strng)*(inventory[ch]-alphFreq[ch]))**2/alphFreq[ch] for ch in inventory]
        return sum(chDegree)
    
    def chiSquaredTest(self) -> list:
        # n = len(alphabet)
        candidates = []
        for a in range(1, n):
            if gcd(a,n) == 1:
                for b in range(n):
                    T = Encipher(self.content,a,b)
                    dec = T.affine(True)
                    candidates.append((a, b, self.chiSquared(dec)))
        return sorted(candidates, key = lambda x: x[2])

In [ ]:
C = ChiSquareAttack('GHIGCTGXAIGPTAXGKGYIOWOIWA')
C.chiSquaredTest()[:3]

In [ ]:
X = Encipher('GHIGCTGXAIGPTAXGKGYIOWOIWA',15,24)

In [ ]:
X.affine()

## Práctica

In [ ]:
def readTxt(file):
    with open(file,'r') as f:
        lines = f.readlines()
    accum = [k[:-1] for k in lines]
    return ''.join(accum)

In [ ]:
E = readTxt('encipheringText.txt')

In [ ]:
print(E)

In [ ]:
C = ChiSquareAttack(E)

In [ ]:
C.chiSquaredTest()[:3]

In [ ]:
P = Encipher(E,21,13)

In [ ]:
P.affine()